<a href="https://colab.research.google.com/github/immaFrogUwU/Samsung-Stock-Price/blob/main/Samsung_stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 005930.KS.csv to 005930.KS.csv


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, when
spark = SparkSession.builder.appName("SamsungStockClean").getOrCreate()

In [ ]:
#Load data
df = spark.read.csv('005930.KS.csv', header=True, inferSchema=True)
df.printSchema()
df.describe().show()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)

+-------+------------------+------------------+-----------------+------------------+------------------+--------------------+
|summary|              Open|              High|              Low|             Close|         Adj Close|              Volume|
+-------+------------------+------------------+-----------------+------------------+------------------+--------------------+
|  count|              1505|              1505|             1505|              1505|              1505|                1505|
|   mean|64154.617940199336| 64734.01993355482|63548.70431893688| 64108.90365448505| 59933.01363164452|1.6625267806644518E7|
| stddev|12269.830762688787|12321.091698838984|12176.81337181051|12227.346772831712|12894.680103072302|   87315

In [ ]:
# Check missing value
for col_name in df.columns:
    df.select(col_name).where(col(col_name).isNull()).show()

# Convert 'Date' to datetime
df = df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd")
)
# Drop rows
df = df.dropna()

# Remove duplicate rows
df = df.dropDuplicates()

# Rename 'Adj Close' column to 'Adj_Close'
df = df.withColumnRenamed("Adj Close", "Adj_Close")

+----+
|Date|
+----+
+----+

+----+
|Open|
+----+
+----+

+----+
|High|
+----+
+----+

+---+
|Low|
+---+
+---+

+-----+
|Close|
+-----+
+-----+

+---------+
|Adj Close|
+---------+
+---------+

+------+
|Volume|
+------+
+------+



In [ ]:
# Check if 'Open' or 'Close' is less than 'Low'
lower_than_low = df.filter((col("Open") < col("Low")) | (col("Close") < col("Low")))
print("Số lần 'Open' hoặc 'Close' thấp hơn 'Low':", lower_than_low.count())
lower_than_low.show()
# Fix incorrect data: if 'Low' < 'Close', set 'Low' to 'Close'
df_fixed = df.withColumn("Low", when(col("Low") < col("Close"), col("Close")).otherwise(col("Low")))

df_fixed.orderBy("Date").show(5)

# Dừng Spark session
spark.stop()

Số lần 'Open' hoặc 'Close' thấp hơn 'Low': 1
+----------+-------+-------+-------+-------+-------------+--------+
|      Date|   Open|   High|    Low|  Close|    Adj_Close|  Volume|
+----------+-------+-------+-------+-------+-------------+--------+
|2024-10-14|59500.0|61200.0|59400.0|59300.0|58898.3984375|20886249|
+----------+-------+-------+-------+-------+-------------+--------+

+----------+-------+-------+-------+-------+---------------+--------+
|      Date|   Open|   High|    Low|  Close|      Adj_Close|  Volume|
+----------+-------+-------+-------+-------+---------------+--------+
|2019-01-02|39400.0|39400.0|38750.0|38750.0| 33061.84765625| 7847664|
|2019-01-03|38300.0|38550.0|37600.0|37600.0|32080.662109375|12471493|
|2019-01-04|37450.0|37600.0|37450.0|37450.0| 31952.66796875|14108958|
|2019-01-07|38000.0|38900.0|38750.0|38750.0| 33061.84765625|12748997|
|2019-01-08|38000.0|39200.0|38100.0|38100.0|32507.259765625|12756554|
+----------+-------+-------+-------+-------+----------